In [49]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [71]:
# import csv file into pandas
# upload to database order (foreign key references require tables be uploaded in ths order): 
# 1: admin_projects
# 2: issues_issue_types
# 3: issues_issue_subtypes
# 4: issues_issues
# note: will be named indicating order

file1 = "data/csv/admin_projects.csv"
file2 = "data/csv/issues_issue_types.csv"
file3 = "data/csv/issues_issue_subtypes.csv"
file4 = "data/csv/issues_issues.csv"
file5 = "static/zip.csv"

csv_data1 = pd.read_csv(file1)
csv_data2 = pd.read_csv(file2)
csv_data3 = pd.read_csv(file3)
csv_data4 = pd.read_csv(file4)
csv_data5 = pd.read_csv(file5)

admin_projects_df_1 = pd.DataFrame(csv_data1)
issues_issue_types_df_2 = pd.DataFrame(csv_data2)
issues_issue_subtypes_df_3 = pd.DataFrame(csv_data3)
issues_issues_df_4 = pd.DataFrame(csv_data4)
zipcodes_as_lat_lon = pd.DataFrame(csv_data5)

#confirm databases are loaded
zipcodes_as_lat_lon.head(5)
#admin_projects_df_1.head(5)
#issues_issue_types_df_2.head(5)
#issues_issue_subtypes_df_3.head(5)
#issues_issues_df_4.head(5)





,00501,40.9223,-72.6371,Holtsville,NY,Suffolk,UNIQUE
0,544,40.9223,-72.6371,Holtsville,NY,Suffolk,UNIQUE
1,601,18.1653,-66.7226,Adjuntas,PR,Adjuntas,STANDARD
2,602,18.3931,-67.1810,Aguada,PR,Aguada,STANDARD
3,603,18.4559,-67.1457,Aguadilla,PR,Aguadilla,STANDARD
4,604,18.4935,-67.1359,Aguadilla,PR,Aguadilla,STANDARD


In [ ]:
# Add Data Munging here

In [69]:
clean_df_1 = admin_projects_df_1[admin_projects_df_1["postal_code"].notna()]
clean_df_1 = clean_df_1[clean_df_1["value"] > 5000000]
clean_df_1 = clean_df_1[clean_df_1["status"] == "active"]
clean_df_1.shape

#drop unneeded rows:
clean_df_1 = clean_df_1[['id','bim360_account_id','name','start_date','type','value','currency','status','job_number','city','state_or_province','postal_code','country','timezone','construction_type','contract_type','business_unit_id','last_sign_in','created_at']]
issues_issue_types_df_2 = issues_issue_types_df_2[['issue_type_id','bim360_account_id','bim360_project_id','issue_type','is_active']]
issues_issue_subtypes_df_3 = issues_issue_subtypes_df_3[['issue_subtype_id','bim360_account_id','bim360_project_id','issue_type_id','issue_subtype']]
issues_issues_df_4 = issues_issues_df_4[['issue_id','bim360_project_id','type_id','subtype_id','status','due_date','assignee_id','root_cause_id']]




#merge tables
#master_df = pd.merge(clean_df_1, issues_issue_types_df_2, left_on=['id', 'bim360_account_id'], right_on=['bim360_project_id', 'bim360_account_id'], how="inner")

#master_df.head(5)


In [42]:
rds_connection_string = "@localhost:5432/project2"

# Important: add password here
password = ""

engine = create_engine(f'postgresql+psycopg2://postgres:{password}{rds_connection_string}')

In [43]:
#confirm database connection and that the tables are present (if not, run schema sql file in database first)
engine.table_names()

['admin_projects',
 'issues_issues',
 'issues_issue_types',
 'issues_issue_subtypes']

In [44]:
#upload data in order specified in beginning


#important note: i removed the foreign key constraints for the time being from the schema for the tables when i
#                ran it in postgres (still in schema file though - at the bottom). Had some issues with FOREIGN
#                KEY CONSTRAINT violations when loading data... missing keys in some tables. so it may be best
#                to run schema without the constraints.


clean_df_1.to_sql(name='admin_projects', con=engine, if_exists='append', index=False)

issues_issue_types_df_2.to_sql(name='issues_issue_types', con=engine, if_exists='append', index=False)

issues_issue_subtypes_df_3.to_sql(name='issues_issue_subtypes', con=engine, if_exists='append', index=False)

issues_issues_df_4.to_sql(name='issues_issues', con=engine, if_exists='append', index=False)

In [ ]:
from sqlalchemy import Table
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.sql.expression import Executable, ClauseElement

class CreateView(Executable, ClauseElement):
    def __init__(self, name, select):
        self.name = name
        self.select = select

@compiles(CreateView)
def visit_create_view(element, compiler, **kw):
    return "CREATE VIEW %s AS %s" % (
         element.name,
         compiler.process(element.select, literal_binds=True)
         )

# test data
from sqlalchemy import MetaData, Column, Integer


metadata = MetaData(engine)
t = Table('t',
          metadata,
          Column('id', Integer, primary_key=True),
          Column('number', Integer))
t.create()
engine.execute(t.insert().values(id=1, number=3))
engine.execute(t.insert().values(id=9, number=-3))

# create view
createview = CreateView('viewname', t.select().where(t.c.id>5))
engine.execute(createview)

# reflect view and print result
v = Table('viewname', metadata, autoload=True)
for r in engine.execute(v.select()):
    print r

In [45]:
#confirm data is loaded

pd.read_sql_query('select * from admin_projects', con=engine).head()

,id,bim360_account_id,name,start_date,end_date,type,value,currency,status,job_number,...,city,state_or_province,postal_code,country,timezone,construction_type,contract_type,business_unit_id,last_sign_in,created_at
0,e5c196de-27ac-43f1-ad9a-e7267316ed9f,49df69ab-c266-4d49-951c-031b16f7a31f,Memorial Stadium,Mon Jan 01 2018 00:00:00 GMT+0000 (Coordinated...,Sun Dec 01 2019 00:00:00 GMT+0000 (Coordinated...,Stadium/Arena,78000000,USD,active,10102939,...,Champaign,IL,61820,US,America/Chicago,New Construction,Design-Bid-Build,1f3be48b-1eef-11e3-89a6-02cc3afa57bd,None,Thu Aug 03 2017 16:33:54 GMT+0000 (Coordinated...
1,ecee877d-e66c-411c-88ef-3b4f0afbfd1f,49df69ab-c266-4d49-951c-031b16f7a31f,10104945 - Brandon Road Relocation,Sat Feb 01 2020 00:00:00 GMT+0000 (Coordinated...,Thu Oct 15 2020 00:00:00 GMT+0000 (Coordinated...,Streets / Roads / Highways,9000000,USD,active,10104945,...,Joliet,IL,60421,US,America/Chicago,New Construction,Design-Bid,869871b3-ac33-4a98-ab47-0ede0ba28d1c,None,Mon Feb 03 2020 13:35:47 GMT+0000 (Coordinated...
2,800194cf-13d3-47df-9ad1-fe5d2a64d6ca,49df69ab-c266-4d49-951c-031b16f7a31f,10103177 - Rockwool Ran5,Mon Sep 03 2018 00:00:00 GMT+0000 (Coordinated...,Fri Apr 10 2020 00:00:00 GMT+0000 (Coordinated...,Manufacturing / Factory,53362230,USD,active,10103177,...,Shenandoah Junction,WV,25442,US,America/New_York,New Construction,Design-Bid,869871b3-ac33-4a98-ab47-0ede0ba28d1c,Mon Feb 10 2020 16:09:36 GMT+0000 (Coordinated...,Tue Jul 31 2018 17:58:10 GMT+0000 (Coordinated...
3,3431b76d-ba63-4c83-8c99-4c88d0ef98a8,49df69ab-c266-4d49-951c-031b16f7a31f,10104980 - Bridgeport Amazon TI,Mon Jan 27 2020 00:00:00 GMT+0000 (Coordinated...,Sat Aug 01 2020 00:00:00 GMT+0000 (Coordinated...,Warehouse (non-manufacturing),58000000,USD,active,10104980,...,Newnan,GA,30263,US,America/New_York,New Construction,Design-Bid,869871b3-ac33-4a98-ab47-0ede0ba28d1c,Fri Feb 07 2020 21:28:24 GMT+0000 (Coordinated...,Mon Feb 03 2020 13:52:57 GMT+0000 (Coordinated...
4,e1b87d21-aef7-4fad-a518-491a4f681463,49df69ab-c266-4d49-951c-031b16f7a31f,00-03925.258-000_SP_Sproul_Hall,Wed May 01 2019 00:00:00 GMT+0000 (Coordinated...,Tue Jun 30 2020 00:00:00 GMT+0000 (Coordinated...,Dormitory,55000000,USD,active,10103710.258,...,University Park,PA,16802,US,America/New_York,Renovation,Design-Bid,35932d9f-1eee-11e3-89a6-02cc3afa57bd,None,Wed Feb 13 2019 13:39:23 GMT+0000 (Coordinated...


In [33]:
clean_df_1.shape

(197, 22)